In [10]:
import random
from dataclasses import dataclass
from typing import List, Tuple

# --- 定数定義 ---
WIDTH, HEIGHT = 10, 10        # 迷路のセル数
START, GOAL   = (1, 1), (8, 8) # 座標系は (x, y)

# --- データ構造 ---
@dataclass
class Maze:
    vert_walls: List[List[bool]]      # 垂直壁 (y, x)  長さ: HEIGHT × (WIDTH-1)
    horiz_walls: List[List[bool]]     # 水平壁 (y, x)  長さ: (HEIGHT-1) × WIDTH

def generate_maze() -> Maze:
    """再帰的バックトラッキングで木構造を生成し、壁の有無で保持する。"""
    visited = [[False]*WIDTH for _ in range(HEIGHT)]
    vert   = [[True]*(WIDTH-1) for _ in range(HEIGHT)]     # True = 壁あり
    horiz  = [[True]*WIDTH      for _ in range(HEIGHT-1)]

    stack: List[Tuple[int, int]] = [START]
    visited[START[1]][START[0]] = True                    # y, x の順に注意

    while stack:
        x, y = stack[-1]

        # 未訪問の隣接セルを列挙
        candidates = []
        if x > 0           and not visited[y][x-1]: candidates.append(('L', x-1, y))
        if x < WIDTH-1     and not visited[y][x+1]: candidates.append(('R', x+1, y))
        if y > 0           and not visited[y-1][x]: candidates.append(('U', x, y-1))
        if y < HEIGHT-1    and not visited[y+1][x]: candidates.append(('D', x, y+1))

        if candidates:
            direction, nx, ny = random.choice(candidates)

            # 壁を破壊
            if direction == 'L':  vert[y][x-1]   = False
            if direction == 'R':  vert[y][x]     = False
            if direction == 'U':  horiz[y-1][x]  = False
            if direction == 'D':  horiz[y][x]    = False

            visited[ny][nx] = True
            stack.append((nx, ny))
        else:
            stack.pop()

    return Maze(vert, horiz)

def render_maze(m: Maze) -> str:
    """壁を #, 通路を空白にして文字列化。"""
    w, h = WIDTH, HEIGHT
    cells = [['#'] * (2*w + 1) for _ in range(2*h + 1)]

    for y in range(h):
        for x in range(w):
            cx, cy = 2*x + 1, 2*y + 1
            cells[cy][cx] = ' '                       # セル本体

            # 左右上下の壁を確認し、無ければ空白を開通
            if x > 0 and not m.vert_walls[y][x-1]: cells[cy][cx-1] = ' '
            if x < w-1 and not m.vert_walls[y][x]: cells[cy][cx+1] = ' '
            if y > 0 and not m.horiz_walls[y-1][x]: cells[cy-1][cx] = ' '
            if y < h-1 and not m.horiz_walls[y][x]: cells[cy+1][cx] = ' '

    # スタートとゴールを目印付きで表示
    sx, sy = START; gx, gy = GOAL
    cells[2*sy+1][2*sx+1] = 'S'
    cells[2*gy+1][2*gx+1] = 'G'

    return '\n'.join(''.join(row) for row in cells)


In [11]:
if __name__ == "__main__":
    maze = generate_maze()
    printable = render_maze(maze)
    print(printable)


#####################
#         #       # #
# ####### ### ### # #
# #S#   # #   # # # #
# # ### # # ### # # #
# #   # # # #     # #
# ### # # # # ##### #
#     #   # # #     #
####### ### # # ### #
#     #     # # #   #
# ### ####### ### # #
#   #       # #   # #
# ### ####### # ### #
# #   #     #   # # #
### ### ### ##### # #
#   #   # #   #     #
# # # ### ### # #####
# # # #     # #  G# #
# ### ##### # ### # #
#           #       #
#####################


In [12]:
def render_maze_compact(m: Maze) -> str:
    """
    壁を次の文字で描画し、行数を (HEIGHT + 1) 行に縮小する。
    ────────────────────────────────────────────────
        '_' : 底辺（水平壁）
        '|' : 右辺（垂直壁）
        ' ' : 通路
        'S','G' : スタート / ゴール
    """
    w, h = WIDTH, HEIGHT
    # 上辺（外周）の水平壁
    lines = [' ' + '_' * w]           # 例)  " ____________"（先頭の空白は左上角の柱）

    for y in range(h):
        row = ['|']                   # 行頭は必ず左外壁
        for x in range(w):
            # デフォルトは通路
            char = ' '
            if (x, y) == START: char = 'S'
            elif (x, y) == GOAL: char = 'G'

            # 下側の水平壁を描画
            if y == h - 1:                            # 最下段は必ず外壁
                char = '_' if char == ' ' else char
            elif m.horiz_walls[y][x]:                 # 内部壁の場合
                char = '_' if char == ' ' else char

            # 右側の垂直壁を描画
            if x == w - 1:                            # 右外周は必ず壁
                row.append(char + '|')
            elif m.vert_walls[y][x]:                  # 内部壁
                row.append(char + '|')
            else:                                     # 壁が無い場合は空白
                row.append(char + ' ')
        lines.append(''.join(row))
    return '\n'.join(lines)


In [13]:
if __name__ == "__main__":
    maze = generate_maze()

    print("◆ 従来のフルスケール表示")
    print(render_maze(maze))

    print("\n◆ コンパクト表示")
    print(render_maze_compact(maze))


◆ 従来のフルスケール表示
#####################
#           #     # #
# ### ##### ### # # #
# #S#     #     # # #
### # ### # ##### # #
#   #   # # #   # # #
# ##### # # # # # # #
# #   # # # # # #   #
# # ### # ### # ### #
# # #   # #   #   # #
# # # ### # ##### ###
# #     #   #   #   #
# ##### ##### # ### #
#     #       #   # #
##### ####### ##### #
#   #     #   #     #
### ##### ##### ### #
#   #   #       #G  #
# ### # ######### ###
#     #             #
#####################

◆ コンパクト表示
 __________
|  _   _ _  |_    | |
|_|S|  _  |  _ _| | |
|  _|_  | | |   | | |
| |  _| | |_| | |_  |
| | |  _| |  _|_  |_|
| |_ _  |_ _|   |_  |
|_ _  |_ _ _  |_ _| |
|_  |_ _  |_ _|  _  |
|  _|   |_ _ _ _|G _|
|_ _ _|_ _ _ _ _ _ _|


In [14]:
def generate_maze_limited() -> Maze:
    """
    直線が 5 マス以上続かないよう制御した DFS 版迷路生成。
    """
    visited = [[False]*WIDTH for _ in range(HEIGHT)]
    vert   = [[True]*(WIDTH-1) for _ in range(HEIGHT)]
    horiz  = [[True]*WIDTH      for _ in range(HEIGHT-1)]

    # スタック要素: (x, y, prev_dir, run_len)
    stack: List[Tuple[int, int, str, int]] = [(START[0], START[1], '', 0)]
    visited[START[1]][START[0]] = True

    while stack:
        x, y, prev_dir, run_len = stack[-1]

        # 未訪問の隣接セルを列挙
        neighbors = []
        if x > 0        and not visited[y][x-1]: neighbors.append(('L', x-1, y))
        if x < WIDTH-1  and not visited[y][x+1]: neighbors.append(('R', x+1, y))
        if y > 0        and not visited[y-1][x]: neighbors.append(('U', x, y-1))
        if y < HEIGHT-1 and not visited[y+1][x]: neighbors.append(('D', x, y+1))

        # 直線 5 マス禁止フィルタ
        if run_len >= 4 and neighbors:
            filtered = [nb for nb in neighbors if nb[0] != prev_dir]
            if filtered:                          # 少なくとも一方向残れば差し替え
                neighbors = filtered

        if neighbors:
            direction, nx, ny = random.choice(neighbors)

            # 壁破壊
            if direction == 'L':  vert[y][x-1]   = False
            if direction == 'R':  vert[y][x]     = False
            if direction == 'U':  horiz[y-1][x]  = False
            if direction == 'D':  horiz[y][x]    = False

            visited[ny][nx] = True
            next_run = run_len + 1 if direction == prev_dir else 1
            stack.append((nx, ny, direction, next_run))
        else:
            stack.pop()

    return Maze(vert, horiz)


In [15]:
if __name__ == "__main__":
    maze = generate_maze_limited()

    print("◆ 従来のフルスケール表示")
    print(render_maze(maze))

    print("\n◆ コンパクト表示")
    print(render_maze_compact(maze))

◆ 従来のフルスケール表示
#####################
#   # #             #
# # # # ### ####### #
# #S#     #   #   # #
# ########### # # # #
# #         #   # # #
# # ####### # ### ###
# # #   #   # # #   #
# # # ### ### # ### #
#   # #   #       # #
##### # ### ####### #
#     # #     #     #
# ### # # ##### ### #
# #   # #   #   #   #
# # ### ##### ### ###
# # #   #     #   # #
# ### ### ##### ### #
#     #   #   #  G# #
# ##### ### # ### # #
#       #   #       #
#####################

◆ コンパクト表示
 __________
|   | |  _   _ _ _  |
| |S|_ _ _|_  |   | |
| |  _ _ _  |  _| |_|
| | |  _|  _| | |_  |
|_ _| |  _|  _ _ _| |
|  _  | |  _ _|  _  |
| |  _| |_ _|  _|  _|
| |_|  _|  _ _|  _| |
|  _ _|  _|   |_ G| |
|_ _ _ _|_ _|_ _ _ _|


In [16]:
import random
from dataclasses import dataclass
from typing import List, Tuple, Set

WIDTH, HEIGHT = 10, 10
START, GOAL   = (1, 1), (8, 8)

@dataclass
class Maze:
    vert_walls:  List[List[bool]]   # (y, x)  ⬅︎ 左がTrueなら壁
    horiz_walls: List[List[bool]]   # (y, x)  ⬅︎ 上がTrueなら壁

# ────────────────────────────────
#  主要ユーティリティ
# ────────────────────────────────
def _cell_neighbors(x: int, y: int) -> List[Tuple[str, int, int]]:
    """上下左右の隣接セルを (方向, nx, ny) で返す（盤外は除外）"""
    neigh = []
    if x > 0:        neigh.append(('L', x-1, y))
    if x < WIDTH-1:  neigh.append(('R', x+1, y))
    if y > 0:        neigh.append(('U', x, y-1))
    if y < HEIGHT-1: neigh.append(('D', x, y+1))
    return neigh

# ────────────────────────────────
#  制約付き DFS 生成
# ────────────────────────────────
def generate_maze_constrained() -> Maze:
    visited = [[False]*WIDTH for _ in range(HEIGHT)]
    degree  = [[0]*WIDTH     for _ in range(HEIGHT)]   # 通路数（壁破壊数）
    vert    = [[True]*(WIDTH-1) for _ in range(HEIGHT)]
    horiz   = [[True]*WIDTH      for _ in range(HEIGHT-1)]

    # スタック要素: (x, y, prev_dir, run_len)
    stack: List[Tuple[int, int, str, int]] = [(START[0], START[1], '', 0)]
    visited[START[1]][START[0]] = True

    while stack:
        x, y, prev_dir, run_len = stack[-1]

        # 未訪問隣接セルを取得
        neighbors = [(d, nx, ny) for d, nx, ny in _cell_neighbors(x, y)
                     if not visited[ny][nx]]

        # 直線 4 マス制限フィルタ
        if run_len >= 4:
            neighbors = [nb for nb in neighbors if nb[0] != prev_dir]

        # 十字交差点禁止フィルタ
        neighbors = [nb for nb in neighbors
                     if degree[ny][nx] < 3 and degree[y][x] < 3]

        if neighbors:
            direction, nx, ny = random.choice(neighbors)

            # --- 壁破壊 ---
            if direction == 'L':  vert[y][x-1]   = False
            if direction == 'R':  vert[y][x]     = False
            if direction == 'U':  horiz[y-1][x]  = False
            if direction == 'D':  horiz[y][x]    = False

            # --- メタ更新 ---
            visited[ny][nx] = True
            degree[y][x]  += 1
            degree[ny][nx] += 1

            next_run = run_len + 1 if direction == prev_dir else 1
            stack.append((nx, ny, direction, next_run))
        else:
            stack.pop()

    return Maze(vert, horiz)


In [17]:
def assert_constraints(m: Maze):
    # 直線長チェック & 十字交差点チェック
    for y in range(HEIGHT):
        for x in range(WIDTH):
            # degree 計算
            deg = 0
            if x > 0     and not m.vert_walls[y][x-1]: deg += 1
            if x < WIDTH-1 and not m.vert_walls[y][x]: deg += 1
            if y > 0     and not m.horiz_walls[y-1][x]: deg += 1
            if y < HEIGHT-1 and not m.horiz_walls[y][x]: deg += 1
            assert deg <= 3, f"cross junction at {(x,y)}"

    # 直線長(横・縦) ≤4
    for y in range(HEIGHT):
        run = 0
        for x in range(WIDTH):
            if x < WIDTH-1 and not m.vert_walls[y][x]:
                run += 1
            else:
                assert run <= 3, "horizontal run >4"
                run = 0
    for x in range(WIDTH):
        run = 0
        for y in range(HEIGHT):
            if y < HEIGHT-1 and not m.horiz_walls[y][x]:
                run += 1
            else:
                assert run <= 3, "vertical run >4"
                run = 0

    # 2×2 通路ループ検出
    for y in range(HEIGHT-1):
        for x in range(WIDTH-1):
            if (not m.vert_walls[y][x]   and not m.vert_walls[y+1][x] and
                not m.horiz_walls[y][x] and not m.horiz_walls[y][x+1]):
                raise AssertionError(f"2x2 open loop at {(x,y)}")


In [18]:
if __name__ == "__main__":
    maze = generate_maze_constrained()
    assert_constraints(maze)          # 失敗時 AssertionError

    print(render_maze_compact(maze))  # 前回のコンパクト描画を再利用


UnboundLocalError: cannot access local variable 'ny' where it is not associated with a value

In [19]:
def generate_maze_constrained() -> Maze:
    """
    制約:
      1) 直線は4マス(=移動3回)以下
      2) 十字交差点(4方向通路)を作らない
      3) 2×2の全開放通路を作らない   …木構造なので自動的に満たされる
    """
    visited = [[False] * WIDTH for _ in range(HEIGHT)]
    degree  = [[0] * WIDTH     for _ in range(HEIGHT)]   # 各セルの通路数
    vert    = [[True] * (WIDTH - 1) for _ in range(HEIGHT)]
    horiz   = [[True] * WIDTH        for _ in range(HEIGHT - 1)]

    stack: List[Tuple[int, int, str, int]] = [(START[0], START[1], '', 0)]
    visited[START[1]][START[0]] = True

    while stack:
        x, y, prev_dir, run_len = stack[-1]

        # ── 未訪問の隣接セル
        neighbors = [
            (d, nx, ny)
            for d, nx, ny in _cell_neighbors(x, y)
            if not visited[ny][nx]
        ]

        # ── 直線4マス制限
        if run_len >= 4:
            neighbors = [nb for nb in neighbors if nb[0] != prev_dir]

        # ── 十字交差点(度数4)禁止
        neighbors = [
            nb for nb in neighbors
            if degree[y][x] < 3 and degree[nb[2]][nb[1]] < 3   # ← 修正点
        ]

        if neighbors:
            direction, nx, ny = random.choice(neighbors)

            # ---- 壁を壊す
            if direction == 'L':  vert[y][x-1]  = False
            if direction == 'R':  vert[y][x]    = False
            if direction == 'U':  horiz[y-1][x] = False
            if direction == 'D':  horiz[y][x]   = False

            # ---- 状態更新
            visited[ny][nx] = True
            degree[y][x]    += 1
            degree[ny][nx]  += 1

            next_run = run_len + 1 if direction == prev_dir else 1
            stack.append((nx, ny, direction, next_run))
        else:
            stack.pop()

    return Maze(vert, horiz)


In [20]:
if __name__ == "__main__":
    maze = generate_maze_constrained()
    assert_constraints(maze)          # 制約を自動チェック
    print(render_maze_compact(maze))  # コンパクト表示


AssertionError: horizontal run >4

In [21]:
def _would_exceed_run(vert: List[List[bool]],
                      horiz: List[List[bool]],
                      x: int, y: int, direction: str) -> bool:
    """
    壁を壊した場合に “同一直線で開通する壁の本数” が 3 を超えるか判定
    （3本＝4セルが上限）
    """
    if direction in ('L', 'R'):
        # --- 水平方向（vert 配列を確認）
        edge_idx = x-1 if direction == 'L' else x           # 壊す予定の垂直壁 index
        # 左側を数える
        run = 1
        i = edge_idx - 1
        while i >= 0 and not vert[y][i]:
            run += 1
            i -= 1
        # 右側を数える
        i = edge_idx + 1
        while i < WIDTH-1 and not vert[y][i]:
            run += 1
            i += 1
        return run > 3
    else:
        # --- 垂直方向（horiz 配列を確認）
        edge_idx = y-1 if direction == 'U' else y           # 壊す予定の水平壁 index
        run = 1
        j = edge_idx - 1
        while j >= 0 and not horiz[j][x]:
            run += 1
            j -= 1
        j = edge_idx + 1
        while j < HEIGHT-1 and not horiz[j][x]:
            run += 1
            j += 1
        return run > 3


def generate_maze_constrained() -> Maze:
    """
    制約:
      ① 直線は4セル以下（連続して開通する壁は最大3本）
      ② 十字交差点（4方向通路）を作らない
      ③ 2×2 全開通領域は生成されない（木構造ゆえ）
    """
    visited = [[False]*WIDTH for _ in range(HEIGHT)]
    degree  = [[0]*WIDTH     for _ in range(HEIGHT)]
    vert    = [[True]*(WIDTH-1) for _ in range(HEIGHT)]
    horiz   = [[True]*WIDTH       for _ in range(HEIGHT-1)]

    stack: List[Tuple[int, int, str, int]] = [(START[0], START[1], '', 0)]
    visited[START[1]][START[0]] = True

    while stack:
        x, y, prev_dir, run_len = stack[-1]

        # ── 未訪問の隣接セル
        neighbors = [
            (d, nx, ny) for d, nx, ny in _cell_neighbors(x, y)
            if not visited[ny][nx]
        ]

        # ── 直線 4セル制限（ローカル run_len での第一次フィルタ）
        if run_len >= 3:
            neighbors = [nb for nb in neighbors if nb[0] != prev_dir]

        filtered = []
        for d, nx, ny in neighbors:
            # 十字交差点禁止
            if degree[y][x] >= 3 or degree[ny][nx] >= 3:
                continue
            # グローバルな直線長チェック
            if _would_exceed_run(vert, horiz, x, y, d):
                continue
            filtered.append((d, nx, ny))
        neighbors = filtered

        if neighbors:
            direction, nx, ny = random.choice(neighbors)

            # --- 壁を壊す
            if direction == 'L':  vert[y][x-1]  = False
            if direction == 'R':  vert[y][x]    = False
            if direction == 'U':  horiz[y-1][x] = False
            if direction == 'D':  horiz[y][x]   = False

            # --- 状態更新
            visited[ny][nx] = True
            degree[y][x]    += 1
            degree[ny][nx]  += 1

            next_run = run_len + 1 if direction == prev_dir else 1
            stack.append((nx, ny, direction, next_run))
        else:
            stack.pop()

    return Maze(vert, horiz)


In [22]:
if __name__ == "__main__":
    maze = generate_maze_constrained()
    assert_constraints(maze)          # すべての制約を自動チェック
    print(render_maze_compact(maze))  # 視認用


 __________
|  _|  _ _  |_    | |
|_ S| |  _|_ _ _| | |
| | | | |  _   _| | |
| |_ _|  _| | |  _ _|
|  _|  _|   |_ _|_  |
|_  |_  |_|   |  _  |
| |   |_ _ _| | |_ _|
|  _|_   _ _| |  _  |
| |_  |_     _|_|G  |
|_ _ _ _|_|_ _ _ _|_|


In [34]:
from collections import deque
from typing import Dict, Tuple, List

TURN_LIMIT = 10   # ← ここで許容する曲がり角の上限を設定

def shortest_path(maze: Maze,
                  start: Tuple[int, int],
                  goal: Tuple[int, int]) -> List[Tuple[int, int]]:
    """壁配列から 4 近傍最短経路を返す（セル座標列）"""
    sx, sy = start; gx, gy = goal
    prev: Dict[Tuple[int, int], Tuple[int, int] | None] = {(sx, sy): None}
    q = deque([(sx, sy)])

    while q:
        x, y = q.popleft()
        if (x, y) == (gx, gy):
            break
        for d, nx, ny in _cell_neighbors(x, y):
            # 壁があればスキップ
            if d == 'L' and maze.vert_walls[y][x-1]: continue
            if d == 'R' and maze.vert_walls[y][x]:   continue
            if d == 'U' and maze.horiz_walls[y-1][x]: continue
            if d == 'D' and maze.horiz_walls[y][x]:   continue
            if (nx, ny) in prev:                    # 既訪問
                continue
            prev[(nx, ny)] = (x, y)
            q.append((nx, ny))

    # ゴールから逆順に辿って経路を復元
    path = []
    cur = (gx, gy)
    while cur is not None:
        path.append(cur)
        cur = prev[cur]
    return list(reversed(path))                     # スタート→ゴール順

def count_turns(path: List[Tuple[int, int]]) -> int:
    """セル列から方向変化回数（＝曲がり角の数）を計算"""
    turns = 0
    if len(path) < 3:
        return 0
    # 最初の方向
    (x0, y0), (x1, y1) = path[0], path[1]
    dx_prev, dy_prev = x1 - x0, y1 - y0
    for (px, py), (qx, qy) in zip(path[1:-1], path[2:]):
        dx, dy = qx - px, qy - py
        if (dx, dy) != (dx_prev, dy_prev):
            turns += 1
        dx_prev, dy_prev = dx, dy
    return turns

def generate_maze_with_turn_limit() -> Maze:
    """従来制約＋曲がり角上限を満たすまで再試行するラッパー"""
    attempt = 0
    while True:
        attempt += 1
        maze = generate_maze_constrained()  # 既存関数（直線≤4 等の制約付き）
        path = shortest_path(maze, START, GOAL)
        turns = count_turns(path)
        if turns <= TURN_LIMIT:
            # print(f"生成試行 {attempt} 回で合格（曲がり {turns} 回）")
            return maze


In [38]:
if __name__ == "__main__":
    maze = generate_maze_with_turn_limit()
    assert_constraints(maze)                 # 既存制約も再確認
    path = shortest_path(maze, START, GOAL)
    print(f"Turn count on shortest path: {count_turns(path)}")
    print(render_maze_compact(maze))


Turn count on shortest path: 9
 __________
|  _ _  |_|_|_ _ _  |
| |S|  _  |    _  | |
| | | | | | | |  _ _|
|_| | | | | | |_|   |
|  _|_  |_ _|_ _ _| |
| |_ _ _  | |  _ _ _|
|_ _ _  | | | |  _  |
|_    | |_ _|_ _|   |
|  _|_|  _ _  |  G| |
|_ _ _ _|_ _ _ _|_ _|


### 追加制約の扱い
1. **直交壁禁止**
   2×2 ブロックを走査し、
   `vert[y][x] & vert[y+1][x] & horiz[y][x] & horiz[y][x+1] == True`
   となる “十字形壁” があれば、4 片のうち 1 片をランダムに除去。
2. **壁直線長 ≤ 4**
   - 垂直壁列・水平壁列を走査し、`True` が 5 本以上連続したら
     4 本ごとに 1 片ずつ除去して分断。
   - 通路は増える方向への操作なので、連通性は必ず保持。


In [39]:
import random
from dataclasses import dataclass
from typing import List, Tuple

WIDTH, HEIGHT = 10, 10
START, GOAL   = (1, 1), (8, 8)

# ─────────────────────────────
#  データ構造
# ─────────────────────────────
@dataclass
class Maze:
    vert_walls:  List[List[bool]]   # True = 壁あり（セル右側）
    horiz_walls: List[List[bool]]   # True = 壁あり（セル下側）

# ─────────────────────────────
#  1) 木構造迷路を普通に生成（DFS）
# ─────────────────────────────
def _cell_neighbors(x: int, y: int) -> List[Tuple[str, int, int]]:
    neigh = []
    if x > 0:        neigh.append(('L', x-1, y))
    if x < WIDTH-1:  neigh.append(('R', x+1, y))
    if y > 0:        neigh.append(('U', x, y-1))
    if y < HEIGHT-1: neigh.append(('D', x, y+1))
    return neigh

def generate_base_maze() -> Maze:
    visited = [[False]*WIDTH for _ in range(HEIGHT)]
    vert    = [[True]*(WIDTH-1) for _ in range(HEIGHT)]
    horiz   = [[True]*WIDTH       for _ in range(HEIGHT-1)]

    stack: List[Tuple[int, int]] = [START]
    visited[START[1]][START[0]] = True

    while stack:
        x, y = stack[-1]
        neigh = [(d, nx, ny) for d, nx, ny in _cell_neighbors(x, y)
                 if not visited[ny][nx]]
        if neigh:
            d, nx, ny = random.choice(neigh)
            if d == 'L': vert[y][x-1]  = False
            if d == 'R': vert[y][x]    = False
            if d == 'U': horiz[y-1][x] = False
            if d == 'D': horiz[y][x]   = False
            visited[ny][nx] = True
            stack.append((nx, ny))
        else:
            stack.pop()
    return Maze(vert, horiz)

# ─────────────────────────────
#  2) 追加制約フィルタ
# ─────────────────────────────
def remove_wall_crossings(m: Maze) -> None:
    """十字形（横2+縦2）壁を崩して直交を回避"""
    changed = True
    while changed:                      # 交差削除で新たに交差が出る可能性に備え loop
        changed = False
        for y in range(HEIGHT-1):
            for x in range(WIDTH-1):
                if (m.vert_walls[y][x] and m.vert_walls[y+1][x] and
                    m.horiz_walls[y][x] and m.horiz_walls[y][x+1]):
                    # 4 片のうち 1 片をランダムに外す
                    choice = random.choice(['v0','v1','h0','h1'])
                    if choice == 'v0': m.vert_walls[y][x]   = False
                    elif choice == 'v1': m.vert_walls[y+1][x] = False
                    elif choice == 'h0': m.horiz_walls[y][x] = False
                    else: m.horiz_walls[y][x+1] = False
                    changed = True

def shorten_wall_runs(m: Maze, max_len: int = 4) -> None:
    """壁が max_len+1 本以上連続したら分断"""
    # ── 水平壁列（horiz）を走査
    for y in range(HEIGHT-1):
        run = 0
        for x in range(WIDTH):
            if m.horiz_walls[y][x]:
                run += 1
                if run > max_len:
                    m.horiz_walls[y][x] = False
                    run = 0
            else:
                run = 0
    # ── 垂直壁列（vert）を走査
    for y in range(HEIGHT):
        run = 0
        for x in range(WIDTH-1):
            if m.vert_walls[y][x]:
                run += 1
                if run > max_len:
                    m.vert_walls[y][x] = False
                    run = 0
            else:
                run = 0

def generate_maze_with_wall_rules() -> Maze:
    m = generate_base_maze()
    remove_wall_crossings(m)   # 直交壁を排除
    shorten_wall_runs(m, 4)    # 5 連以上の壁を分断
    return m


In [40]:
def render_maze_compact(m: Maze) -> str:
    w, h = WIDTH, HEIGHT
    lines = [' ' + '_' * w]
    for y in range(h):
        row = ['|']
        for x in range(w):
            # セル内表示
            ch = ' '
            if (x, y) == START: ch = 'S'
            elif (x, y) == GOAL: ch = 'G'
            # 下壁
            if y == h-1 or m.horiz_walls[y][x]:
                ch = '_' if ch == ' ' else ch
            # 右壁
            if x == w-1 or m.vert_walls[y][x]:
                row.append(ch + '|')
            else:
                row.append(ch + ' ')
        lines.append(''.join(row))
    return '\n'.join(lines)

# ── 動作確認 ─────────────────────────────────────
if __name__ == "__main__":
    maze = generate_maze_with_wall_rules()
    print(render_maze_compact(maze))


 __________
|   |  _  |  _|  _  |
| |S| |  _|_ _  |  _|
| |_  |_ _ _  |_|_  |
|_ _ _| |  _ _|  _  |
| |  _ _ _|_   _|  _|
| | |_   _ _ _|  _| |
|  _|  _|  _ _|_  | |
| |  _|  _|  _  |_  |
|_ _| |   | | |  G| |
|_ _ _ _|_ _ _|_ _ _|
